In [1]:
%%sh
echo '{"kernel_python_credentials" : {"url": "http://ip-172-31-32-142.ec2.internal:8998/"}, "session_configs": 
{"executorMemory": "2g","executorCores": 2,"numExecutors":4}}' > ~/.sparkmagic/config.json
less ~/.sparkmagic/config.json

sh: line 2: /home/emr-notebook/.sparkmagic/config.json: No such file or directory
/home/emr-notebook/.sparkmagic/config.json: No such file or directory


In [1]:
%%configure -f
{
    "conf":  { 
             "spark.jars.packages":"org.apache.iceberg:iceberg-spark3-runtime:0.11.0",
             "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
             "spark.sql.catalog.spark_catalog":"org.apache.iceberg.spark.SparkSessionCatalog",
             "spark.sql.catalog.spark_catalog.type":"hive",
             "spark.sql.catalog.local":"org.apache.iceberg.spark.SparkCatalog",
             "spark.sql.catalog.local.type":"hadoop",
             "spark.sql.catalog.local.warehouse":"s3://vasveena-test-demo/iceberg/catalog/tables/"
           } 
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1619633879001_0008,pyspark,busy,Link,Link,


In [2]:
import time

config = {
    "table_name": "example_iceberg_perf_test_4"
}

spark.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1619633879001_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'3.0.1-amzn-0'

## Read the Data

Let's read our 100M record dataset:

In [3]:
input_df = spark.read.parquet("s3://neilawstmp2/tmp/hudi-perf/input/")
input_df.count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100000000

In [4]:
input_df.createOrReplaceTempView("inputdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
input_df.printSchema()
input_df.rdd.getNumPartitions()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: long (nullable = true)
 |-- month: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: struct (nullable = true)
 |    |-- key1: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- year: string (nullable = true)
 |-- modified_timestamp: timestamp (nullable = true)

25

In [6]:
%%sh
aws s3 ls s3://neilawstmp2/tmp/hudi-perf/input/ | wc -l

38


## Write Parquet

In [6]:
from pyspark.sql.functions import *

input_df=input_df.withColumn("z",expr("substr(md5(concat(id)),1,1)"))
input_df=input_df.withColumn('schema-v',lit('v1'))
input_df=input_df.withColumn('data-v',lit('v2'))
input_df=input_df.withColumn('trade_dt',expr("substr(modified_timestamp,1,10)"))
input_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|     id|month|     sk|txt|                uuid|year| modified_timestamp|  z|schema-v|data-v|  trade_dt|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
|4000000|    3|4000000|[E]|6e505939-f5fd-4ab...|2019|2021-04-02 00:05:02|  9|      v1|    v2|2021-04-02|
|4000001|    9|4000001|[F]|20486aca-2759-43f...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000002|   11|4000002|[G]|42962a21-a2dc-40d...|2019|2021-04-02 00:05:02|  d|      v1|    v2|2021-04-02|
|4000003|    9|4000003|[H]|9841ad6d-1532-496...|2019|2021-04-02 00:05:02|  c|      v1|    v2|2021-04-02|
|4000004|    4|4000004|[I]|ff1a855a-cced-495...|2019|2021-04-02 00:05:02|  4|      v1|    v2|2021-04-02|
+-------+-----+-------+---+--------------------+----+-------------------+---+--------+------+----------+
only showing top 5 rows

In [9]:
#s3_location=f"s3://neilawstmp2/tmp/parquet-perf/catalog/{config['table_name']}"

s3_location=f"s3://vasveena-test-demo/temp6/parquet-perf/catalog/{config['table_name']}"

partition_cols=['z','schema-v','data-v','trade_dt']
input_df.write.mode("OVERWRITE").partitionBy(partition_cols).parquet(s3_location)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Bulk Insert in Iceberg

Now let's write the same out in Iceberg:

In [7]:
import time
#s3_location=f"s3://neilawstmp2/iceberg/tables/catalog/{config['table_name']}"
#data_location="s3://neilawstmp2/iceberg/tables/data/"

s3_location=f"s3://vasveena-test-demo/iceberg/tables/catalog/{config['table_name']}"
data_location="s3://vasveena-test-demo/iceberg/tables/data2/"

    
start = time.time()

partition_cols=['year']
input_df.write.mode("OVERWRITE")\
.option('path',s3_location) \
.option('write.object-storage.enabled','true')\
.option('write.object-storage.path',data_location)\
.partitionBy(partition_cols).format("iceberg")\
.saveAsTable(config["table_name"])

print(f"Executed in : {time.time() - start}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executed in : 97.53534412384033

In [ ]:
spark.sql("")

In [8]:
spark.sql("SHOW TABLES").show(10,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------------------+-----------+
|database|tableName                  |isTemporary|
+--------+---------------------------+-----------+
|default |example_iceberg_perf_test_1|false      |
|default |example_iceberg_perf_test_4|false      |
|default |table_sample_iceberg4      |false      |
|default |table_sample_iceberg66     |false      |
|        |inputdf                    |true       |
+--------+---------------------------+-----------+

In [18]:
spark.sql("DROP TABLE example_iceberg_perf_test").show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

In [74]:
spark.sql("select count(1) from example_iceberg_perf_test LIMIT 10").show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
| count(1)|
+---------+
|100000000|
+---------+

In [75]:
spark.sql("SHOW CREATE TABLE example_iceberg_perf_test").show(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Failed to execute SHOW CREATE TABLE against table `default`.`example_iceberg_perf_test`, which is created by Hive and uses the following unsupported serde configuration
 SERDE: org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe INPUTFORMAT: org.apache.hadoop.mapred.FileInputFormat OUTPUTFORMAT: org.apache.hadoop.mapred.FileOutputFormat;
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 649, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 134, in deco
    raise_from(converted)
  File "<string>", line 3, in raise_from
pyspark.sql.utils.AnalysisException: Failed to execute SHOW CREATE TABLE against table `default`.`example_iceberg_perf_test`, w

In [21]:
%%sh
aws s3 ls s3://neilawstmp2/tmp/iceberg-perf/catalog/example_iceberg_perf_test/data/ | wc -l

1


In [22]:
%%sh
aws s3 ls s3://neilawstmp2/tmp/iceberg-perf/catalog/example_iceberg_perf_test/

                           PRE data/
                           PRE metadata/


In [23]:
%%sh
aws s3 ls s3://neilawstmp2/tmp/iceberg-perf/catalog/example_iceberg_perf_test/data/year=2019/

2021-04-22 19:52:55  129152558 00000-25-3dfc4b64-a6cb-4662-9686-566b1c9bf8da-00001.parquet
2021-04-22 19:52:55  129150847 00001-26-b9a44a7d-0dfd-47fd-85a3-38538b20355e-00001.parquet
2021-04-22 19:52:58  129155524 00002-27-e22f0c01-ff59-4a2e-9cd1-4bc6d375df27-00001.parquet
2021-04-22 19:52:58  129154578 00003-28-eb503339-2986-4c29-b0e1-4487e3cf9555-00001.parquet
2021-04-22 19:52:57  126367557 00004-29-18675f0e-73ea-40a7-bfa4-813822b82656-00001.parquet
2021-04-22 19:52:50  101098090 00005-30-1dca0f83-361b-4fa0-89ea-690e6d57e46c-00001.parquet
2021-04-22 19:52:50  101097695 00006-31-2026de01-a8ef-486a-a8f7-fc9f2dd321e2-00001.parquet
2021-04-22 19:52:50  101097023 00007-32-fe028d46-dd64-4fce-94ba-bff641328b42-00001.parquet
2021-04-22 19:52:49  101094489 00008-33-35923c84-1670-4701-8930-10e9d295fdec-00001.parquet
2021-04-22 19:52:41   75824046 00009-34-ef522a10-51d2-4abc-b445-c5080d916cf8-00001.parquet
2021-04-22 19:52:42   75823811 00010-35-95a999c5-f679-4bf6-bd57-58b1c1a0b582-00001.parquet

In [9]:
spark.sql("DROP TABLE default.table_sample_iceberg66").show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

++
||
++
++

In [11]:
spark.sql("CREATE TABLE table_sample_iceberg6666 (id bigint,data string,category string,ts timestamp,year string) \
USING iceberg \
OPTIONS ( \
    'write.object-storage.enabled'=true,  \
    'write.object-storage.path'='s3://vasveena-test-demo/iceberg/tables/data6666/') \
PARTITIONED BY (year) \
LOCATION 's3://vasveena-test-demo/temp5/parquet-perf/catalog/example_iceberg_perf_test_6666'")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [12]:
spark.sql("SHOW TABLES").show(10,False)
input_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------------------+-----------+
|database|tableName                  |isTemporary|
+--------+---------------------------+-----------+
|default |example_iceberg_perf_test_1|false      |
|default |example_iceberg_perf_test_4|false      |
|default |table_sample_iceberg4      |false      |
|default |table_sample_iceberg6666   |false      |
|        |inputdf                    |true       |
+--------+---------------------------+-----------+

root
 |-- id: long (nullable = true)
 |-- month: long (nullable = true)
 |-- sk: long (nullable = true)
 |-- txt: struct (nullable = true)
 |    |-- key1: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- year: string (nullable = true)
 |-- modified_timestamp: timestamp (nullable = true)
 |-- z: string (nullable = true)
 |-- schema-v: string (nullable = false)
 |-- data-v: string (nullable = false)
 |-- trade_dt: string (nullable = true)

In [13]:
spark.sql("insert overwrite table table_sample_iceberg6666 partition (year) select id,month,uuid,modified_timestamp,year from inputdf")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [10]:
import org.apache.iceberg.spark.IcebergSpark
import org.apache.spark.sql.types.DataTypes

//IcebergSpark.registerBucketUDF(spark, "iceberg_bucket16", DataTypes.LongType, 16)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
<console>:23: error: object iceberg is not a member of package org.apache
       import org.apache.iceberg.spark.IcebergSpark
                         ^



In [39]:
spark.sql("INSERT INTO table_sample_iceberg \
SELECT id, month, year, modified_timestamp FROM example_iceberg_perf_test")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o84.sql.
: org.apache.spark.SparkException: Writing job aborted.
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:413)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:361)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.writeWithV2(WriteToDataSourceV2Exec.scala:253)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.run(WriteToDataSourceV2Exec.scala:259)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:39)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result(V2CommandExec.scala:39)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.executeCollect(V2CommandExec.scala:45)
	at org.apache.spark.sql.Dataset.$anonfun$logicalPlan$1(Dataset.scala:230)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$

In [4]:
%load_ext sparksql_magic

ModuleNotFoundError: No module named 'sparksql_magic'